In [1]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [3]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [4]:
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
from IPython import display
display.clear_output()
from IPython.display import display, Image


In [2]:
# Custom Dataset
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="9utz67dMM55OxuIj31rU")
project = rf.workspace("chickenstrike").project("chickenstrike")
version = project.version(2)
dataset = version.download("yolov5-obb")



/content/datasets
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ChickenStrike-2 in yolov5-obb:: 100%|██████████| 56/56 [00:00<00:00, 2412.03it/s]


In [11]:
# Format data so it will draw oriented bounding boxes
import os

# Define classes and indices
class_names = ['chicken','feeder']
class_indices = {name: index for index, name in enumerate(class_names)} # need this because we need indices

# Image dimensions
img_width, img_height = 640, 640

# Path to folder containing annotation files
# Also need to do for test and valid
folder_path = '/content/datasets/ChickenStrike-2/train/labels'

# Function to normalize coordinates
def normalize_coordinates(coord, max_value):
    return float(coord) / max_value

# Process each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):  # assuming the annotation files are .txt files
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            lines = file.readlines()


        new_lines = []
        for line in lines:
            parts = line.strip().split(' ')
            if len(parts) == 10:
                # Extract label and coordinates
                label = parts[-2]
                coords = parts[:8]

                # Get the class index
                class_index = class_indices.get(label, -1)
                if class_index != -1:
                    # Normalize coordinates
                    normalized_coords = [normalize_coordinates(coords[i], img_width if i % 2 == 0 else img_height) for i in range(8)]

                    # Convert to the desired format and add to new lines
                    new_line = f"{class_index} " + " ".join(map(str, normalized_coords))
                    new_lines.append(new_line)

        # Write the converted lines to a new file or overwrite the existing file
        with open(file_path, 'w') as file:
            file.write('\n'.join(new_lines))

In [13]:
# Train Model
%cd {HOME}

!yolo task=obb mode=train model=yolov8s-obb.pt data={dataset.location}/data.yaml epochs=50 imgsz=640 batch=8

/content
Ultralytics YOLOv8.2.74 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=obb, mode=train, model=yolov8s-obb.pt, data=/content/datasets/ChickenStrike-2/data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=

In [16]:
# Validate Model
%cd {HOME}

!yolo task=obb mode=val model={HOME}/runs/obb/train2/weights/best.pt data={dataset.location}/data.yaml

/content
Ultralytics YOLOv8.2.74 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s-obb summary (fused): 187 layers, 11,412,345 parameters, 0 gradients, 29.4 GFLOPs
val: Scanning /content/datasets/ChickenStrike-2/valid/labels.cache... 5 images, 0 backgrounds, 0 corrupt: 100% 5/5 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 1/1 [00:00<00:00,  1.89it/s]
                   all          5        133      0.947      0.918      0.962       0.77
               chicken          5        128      0.964      0.836       0.93      0.677
                feeder          5          5      0.931          1      0.995      0.864
Speed: 0.4ms preprocess, 29.9ms inference, 0.0ms loss, 54.4ms postprocess per image
Results saved to runs/obb/val
💡 Learn more at https://docs.ultralytics.com/modes/val


In [17]:
# Test Model
%cd {HOME}
!yolo task=obb mode=predict model={HOME}/runs/obb/train2/weights/best.pt conf=0.5 source={dataset.location}/test/images save=true

/content
Ultralytics YOLOv8.2.74 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s-obb summary (fused): 187 layers, 11,412,345 parameters, 0 gradients, 29.4 GFLOPs

image 1/12 /content/datasets/ChickenStrike-2/test/images/chicken_group_10_png.rf.60ca04f75a7f52d7f45e48ac26ae9cfc.jpg: 640x640 18.4ms
image 2/12 /content/datasets/ChickenStrike-2/test/images/chicken_group_11_png.rf.f226d83a5a6e49795282f6beefc1e621.jpg: 640x640 17.3ms
image 3/12 /content/datasets/ChickenStrike-2/test/images/chicken_group_1_png.rf.378809dde4b6bd89af107cb64e5dce15.jpg: 640x640 17.3ms
image 4/12 /content/datasets/ChickenStrike-2/test/images/chicken_group_2_png.rf.baa25f72cb1d613b606da0b6a30ea615.jpg: 640x640 17.2ms
image 5/12 /content/datasets/ChickenStrike-2/test/images/chicken_group_3_png.rf.bb5877a22a36e0252755d9a8f3e6334a.jpg: 640x640 17.4ms
image 6/12 /content/datasets/ChickenStrike-2/test/images/chicken_group_4_png.rf.e7c24eff1361c58d860debe99528e02a.jpg: 640x640 17.2ms
image 7/12 /co

In [ ]:
model = YOLO(f'{HOME}/runs/obb/train2/weights/best.pt')
results = model('chickens_shorter.mp4', show=True, save=True)